In [65]:
#import copy
import sys
import numpy as np
import open3d as o3d
sys.path.append(".")
import pyransac3d as pyrsc
import flattten_pcd as fp
import matplotlib.pyplot as plt
pcd = o3d.io.read_point_cloud("point_pipe.pcd")
print("length pcd ", len(np.array(pcd.points)))
o3d.visualization.draw_geometries([pcd])

MaxX = 4
MinX = 0
MaxZ = 3
MinZ = 0
MaxR = 1
thresh=0.01
maxIteration=10000
voxel_size=0.01

length pcd  1826


In [66]:

print("Downsample the point cloud with a voxel of ", voxel_size)
pcd2 = pcd.voxel_down_sample(voxel_size)
#print("length down sampling pcd ", len(np.array(pcd2.points)))
#o3d.visualization.draw_geometries([pcd2])
plane_model, inliers = pcd2.segment_plane(distance_threshold=0.1,
                                        ransac_n=3,
                                        num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")


Downsample the point cloud with a voxel of  0.01
Plane equation: 0.20x + -0.08y + 0.98z + -0.57 = 0


In [67]:
inlier_cloud = pcd2.select_by_index(inliers)
print(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd2.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0, 1.0, 0])
print("inlider")
o3d.visualization.draw_geometries([inlier_cloud ,outlier_cloud])
print("outlier_cloud")
o3d.visualization.draw_geometries([outlier_cloud])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 19, 21, 22, 23, 24, 25, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 92, 95, 97, 98, 99, 100, 101, 102, 103, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 132, 134, 135, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 178, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 212, 213, 214, 215, 216, 217, 218, 219, 221, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 237, 239, 240, 242, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 25

In [68]:
if inlier_cloud == np.NaN:
        outlier_cloud = pcd2
outlier_cloud = inlier_cloud
print(len(inliers), len(np.array(pcd2.points)))
print("inlider")
#o3d.visualization.draw_geometries([inlier_cloud])
print("outlier_cloud")
o3d.visualization.draw_geometries([outlier_cloud])
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.1, min_points=10, print_progress=True))


958 1132
inlider
outlier_cloud
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2                       ] 5%
Precompute neighbors.[========================================] 100%


In [69]:
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
#print("length label:  ",len(labels))
#print("length outlier_cloud:  ",len(np.array(outlier_cloud.points)))
center = []
normal = []
radius = []

point cloud has 2 clusters


In [71]:
for c in range(max_label+1):
    print(c)
    cluster = [i for i, n in enumerate(labels.tolist()) if n == c]
    print(cluster) 
    cluster = outlier_cloud.select_by_index(cluster)
    print("length cluster:  ",len(np.array(cluster.points)))
    #o3d.visualization.draw_geometries([cluster])
    #o3d.visualization.draw_geometries([outlier_cloud])
    cil = pyrsc.Cylinder()
    points = np.asarray(cluster.points)
    cen, nor, rad, inliers = cil.fit(points, thresh, maxIteration)
    mesh_cylinder = fp.get_clylinder_mesh(cen, nor, rad)
    print("center: " + str(cen))
    print("radius: " + str(rad))
    print("vecC: " + str(nor))
    o3d.visualization.draw_geometries([mesh_cylinder,inlier_cloud],
                                      mesh_show_wireframe=True, mesh_show_back_face=False)
    if (rad < MaxR and cen[0] > MinX and cen[0] < MaxX and cen[2] > MinZ and cen[2] < MaxZ):
        center.append(cen)
        normal.append(nor)
        radius.append(rad)


0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 24, 25, 26, 27, 31, 32, 33, 38, 45, 51, 53, 54, 55, 56, 57, 58, 60, 61, 62, 64, 78, 79, 80, 81, 82, 83, 85, 89, 93, 95, 97, 102, 106, 107, 108, 109, 114, 118, 119, 125, 127, 131, 135, 136, 137, 138, 141, 143, 144, 145, 150, 151, 153, 156, 160, 162, 164, 171, 172, 173, 174, 176, 177, 179, 180, 181, 182, 183, 184, 185, 187, 188, 190, 192, 194, 196, 198, 199, 200, 201, 204, 206, 208, 210, 211, 213, 214, 216, 217, 219, 220, 223, 229, 231, 240, 244, 248, 252, 254, 256, 257, 261, 262, 264, 265, 268, 276, 277, 280, 282, 286, 289, 291, 295, 297, 300, 304, 307, 308, 309, 311, 313, 314, 315, 316, 317, 320, 321, 324, 326, 329, 330, 332, 334, 336, 339, 345, 348, 352, 353, 358, 363, 367, 370, 372, 376, 378, 379, 381, 382, 384, 387, 388, 389, 390, 391, 392, 393, 396, 398, 402, 408, 409, 413, 416, 418, 420, 421, 425, 426, 429, 432, 434, 438, 439, 440, 444, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 460, 461, 462, 463, 465, 466, 474, 475, 478, 480, 4